# Análisis de OEE en la Industria del Empaque
## Notebook 1 — Exploración y Limpieza de Datos

Este notebook cubre las primeras tres fases del proceso de análisis de datos según el marco del **Certificado de Análisis de Datos de Google**:

1. **Ask** — Definir la pregunta de negocio
2. **Prepare** — Describir y evaluar las fuentes de datos
3. **Process** — Limpiar y transformar los datos

---

## Fase 1 — Ask (Pregunta de Negocio)

### ¿Cuál es el problema a resolver?
Una planta de empaque cuenta con 5 máquinas empacadoras que registran datos de telemetría continuamente. La gerencia de operaciones necesita entender **por qué las máquinas no están produciendo al máximo de su capacidad** y dónde están las principales oportunidades de mejora.

### Tarea empresarial (Business Task)
> Analizar los datos de telemetría de las máquinas empacadoras para calcular el **OEE (Overall Equipment Effectiveness)** de cada equipo, identificar las causas principales de tiempo perdido, y generar recomendaciones accionables para mejorar la eficiencia de la planta.

### Métricas clave
- **Disponibilidad (Availability)**: % del tiempo planeado que la máquina estuvo operando.
- **Rendimiento (Performance)**: velocidad real vs. velocidad nominal.
- **Calidad (Quality)**: piezas buenas (output `po`) vs. piezas totales (input `pi`).
- **OEE** = Disponibilidad × Rendimiento × Calidad.

### Stakeholders
- Gerente de operaciones de la planta.
- Equipo de mantenimiento.
- Dirección general (resumen ejecutivo).

---
## Fase 2 — Prepare (Fuentes de Datos)

### Descripción del dataset
Se utiliza el **"Packaging Industry Anomaly Detection Dataset"** disponible públicamente en [Kaggle](https://www.kaggle.com/datasets/orvile/packaging-industry-anomaly-detection-dataset). El dataset contiene dos archivos:

| Archivo | Descripción | Tamaño aprox. |
|---------|-------------|---------------|
| `raw_data.csv` | Registro de cada evento/estado de las máquinas con timestamps, tipo de estado, duración y contadores de producción | ~49 MB |
| `sequences_1h_data.csv` | Resumen agregado por hora con porcentajes de estado y conteos de alarmas | ~11 MB |

### Credibilidad y limitaciones
- **Fuente**: dataset público en Kaggle, recopilado de sensores industriales reales.
- **Cobertura temporal**: datos del año 2020.
- **Equipos**: 5 máquinas identificadas como `s_1` a `s_5`.
- **Limitación**: no se cuenta con datos de contexto (turnos, operadores, tipo de producto).

### 2.1 Carga y exploración inicial de `raw_data.csv`

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuración visual
sns.set_theme(style='whitegrid')

# Cargar el archivo principal
df_raw = pd.read_csv('../datos/raw_data.csv')

print(f"Dimensiones del dataset: {df_raw.shape[0]:,} filas × {df_raw.shape[1]} columnas")
print(f"\nColumnas: {list(df_raw.columns)}")
df_raw.head()

Dimensiones del dataset: 429,394 filas × 10 columnas

Columnas: ['interval_start', 'equipment_ID', 'alarm', 'type', 'start', 'end', 'elapsed', 'pi', 'po', 'speed']


,interval_start,equipment_ID,alarm,type,start,end,elapsed,pi,po,speed
0,2020-01-01 11:21:28.907000+00:00,s_1,A_000,scheduled_downtime,1.577878e+09,1.577878e+09,63050,59916598,59517799,0
1,2020-01-01 11:22:31.957000+00:00,s_1,A_000,idle,1.577878e+09,1.577878e+09,30840,59916598,59517799,0
2,2020-01-01 11:23:02.797000+00:00,s_1,A_000,scheduled_downtime,1.577878e+09,1.577879e+09,1410671,59916598,59517799,0
3,2020-01-01 11:46:33.468000+00:00,s_1,A_000,idle,1.577879e+09,1.577881e+09,1524520,59916598,59517799,0
4,2020-01-01 12:11:57.988000+00:00,s_1,A_000,scheduled_downtime,1.577881e+09,1.577884e+09,2831270,59916598,59517799,0


In [4]:
# Información general del dataset
print("--- Tipos de datos e información general ---")
df_raw.info()

--- Tipos de datos e información general ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429394 entries, 0 to 429393
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   interval_start  429394 non-null  object 
 1   equipment_ID    429394 non-null  object 
 2   alarm           429394 non-null  object 
 3   type            429394 non-null  object 
 4   start           429394 non-null  float64
 5   end             429394 non-null  float64
 6   elapsed         429394 non-null  int64  
 7   pi              429394 non-null  int64  
 8   po              429394 non-null  int64  
 9   speed           429394 non-null  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 32.8+ MB


In [5]:
# Valores nulos por columna
nulos = df_raw.isnull().sum()
print("--- Valores nulos por columna ---")
print(nulos[nulos > 0] if nulos.sum() > 0 else "No se encontraron valores nulos.")

--- Valores nulos por columna ---
No se encontraron valores nulos.


In [6]:
# Estadísticas descriptivas de las columnas numéricas
print("--- Estadísticas Descriptivas ---")
df_raw.describe()

--- Estadísticas Descriptivas ---


,start,end,elapsed,pi,po,speed
count,4.293940e+05,4.293940e+05,4.293940e+05,4.293940e+05,4.293940e+05,429394.000000
mean,1.613331e+09,1.613331e+09,3.699865e+05,3.119454e+07,3.460735e+07,5097.274184
std,1.930947e+07,1.930949e+07,1.368095e+06,3.432799e+07,3.410896e+07,1403.143281
min,1.577837e+09,1.577837e+09,1.140000e+02,0.000000e+00,0.000000e+00,0.000000
25%,1.595056e+09,1.595057e+09,1.392000e+04,8.433338e+05,1.031434e+06,4500.000000
50%,1.619039e+09,1.619039e+09,6.429400e+04,1.123067e+07,2.600625e+07,5526.000000
75%,1.629970e+09,1.629970e+09,2.402800e+05,6.622470e+07,6.579510e+07,6176.000000
max,1.641080e+09,1.641084e+09,7.929008e+07,3.385590e+08,3.391488e+08,6500.000000


In [7]:
# Distribución de estados (tipos de evento)
print("--- Distribución de tipos de estado ---")
print(df_raw['type'].value_counts())
print(f"\nEquipos únicos: {df_raw['equipment_ID'].unique()}")

--- Distribución de tipos de estado ---
type
production            146795
performance_loss      118584
downtime               92084
idle                   50149
scheduled_downtime     21782
Name: count, dtype: int64

Equipos únicos: ['s_1' 's_2' 's_3' 's_4' 's_5']


### 2.2 Carga y exploración inicial de `sequences_1h_data.csv`

Este archivo contiene datos agregados por hora con información adicional como porcentajes de tiempo en cada estado y conteo de alarmas.

In [8]:
# Cargar archivo de secuencias por hora
df_seq = pd.read_csv('../datos/sequences_1h_data.csv')

print(f"Dimensiones: {df_seq.shape[0]:,} filas × {df_seq.shape[1]} columnas")
df_seq.head(3)

Dimensiones: 23,376 filas × 164 columnas


,interval_start,equipment_ID,count_sum,A_028,A_029,A_024,A_045,A_001,A_058,A_064,...,A_069,A_072,A_047,A_059,A_046,A_063,A_053,A_054,A_060,A_061
0,2020-01-01 14:00:00,s_1,4,0.0,0,0,NaN,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01 15:00:00,s_1,2,0.0,0,0,NaN,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 17:00:00,s_1,1,0.0,0,0,NaN,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Columnas clave del archivo de secuencias
cols_clave = ['interval_start', 'equipment_ID', '%idle', '%production', 
              '%downtime', '%performance_loss', '%scheduled_downtime', '#changes']
print("--- Columnas clave del archivo de secuencias ---")
df_seq[cols_clave].describe()

--- Columnas clave del archivo de secuencias ---


,%idle,%production,%downtime,%performance_loss,%scheduled_downtime,#changes
count,23376.000000,23376.000000,23376.000000,23376.000000,23376.000000,23376.000000
mean,0.068730,0.631779,0.132762,0.131189,0.035539,14.316735
std,0.160961,0.279628,0.162798,0.169048,0.103837,12.433900
min,0.000000,0.000000,0.000438,0.000000,0.000000,0.000000
25%,0.000000,0.456896,0.029329,0.005837,0.000000,6.000000
50%,0.000000,0.702421,0.078096,0.067561,0.000000,11.000000
75%,0.049836,0.857945,0.171148,0.184372,0.014923,19.000000
max,0.996116,0.999512,1.000000,0.992402,0.986039,197.000000


---
## Fase 3 — Process (Limpieza y Transformación)

En esta sección se documentan todos los pasos de limpieza realizados sobre los datos.

### 3.1 Conversión de fechas

La columna `interval_start` viene como texto. La convertimos a formato `datetime` de pandas para poder realizar análisis temporales (agrupaciones por día, semana, etc.).

In [10]:
# Convertir interval_start a datetime
df_raw['interval_start'] = pd.to_datetime(df_raw['interval_start'], format='mixed')

print(f"Rango temporal: {df_raw['interval_start'].min()} → {df_raw['interval_start'].max()}")
print(f"Tipo de dato después de conversión: {df_raw['interval_start'].dtype}")

Rango temporal: 2020-01-01 00:05:02.863000+00:00 → 2022-01-01 23:41:35.677000+00:00
Tipo de dato después de conversión: datetime64[ns, UTC]


### 3.2 Verificación de duplicados

In [11]:
# Verificar filas duplicadas
duplicados = df_raw.duplicated().sum()
print(f"Filas duplicadas encontradas: {duplicados}")

if duplicados > 0:
    df_raw = df_raw.drop_duplicates()
    print(f"Duplicados eliminados. Nuevas dimensiones: {df_raw.shape}")

Filas duplicadas encontradas: 0


### 3.3 Estandarización de nombres

Se crean columnas con nombres descriptivos en español para facilitar la lectura en los gráficos y en el reporte final.

In [12]:
# Diccionario para traducir tipos de estado
traduccion_estados = {
    'idle': 'Inactividad (Idle)',
    'scheduled_downtime': 'Mantenimiento Planeado',
    'performance_loss': 'Pérdida de Rendimiento',
    'downtime': 'Falla de Equipo (Downtime)',
    'production': 'Producción'
}

# Diccionario para renombrar equipos
nombres_equipos = {
    's_1': 'Máquina 1', 's_2': 'Máquina 2', 's_3': 'Máquina 3',
    's_4': 'Máquina 4', 's_5': 'Máquina 5'
}

df_raw['tipo_estado'] = df_raw['type'].map(traduccion_estados).fillna(df_raw['type'])
df_raw['equipo_nombre'] = df_raw['equipment_ID'].map(nombres_equipos).fillna(df_raw['equipment_ID'])

print("--- Tipos de estado (traducidos) ---")
print(df_raw['tipo_estado'].value_counts())
print(f"\n--- Equipos (renombrados) ---")
print(df_raw['equipo_nombre'].value_counts())

--- Tipos de estado (traducidos) ---
tipo_estado
Producción                    146795
Pérdida de Rendimiento        118584
Falla de Equipo (Downtime)     92084
Inactividad (Idle)             50149
Mantenimiento Planeado         21782
Name: count, dtype: int64

--- Equipos (renombrados) ---
equipo_nombre
Máquina 1    127676
Máquina 5    100855
Máquina 4     96644
Máquina 2     69765
Máquina 3     34454
Name: count, dtype: int64


### 3.4 Validación de columnas numéricas

Verificamos que no haya valores negativos en columnas que deberían ser siempre positivas (`elapsed`, `pi`, `po`, `speed`).

In [13]:
# Verificar valores negativos en columnas clave
cols_positivas = ['elapsed', 'pi', 'po', 'speed']
for col in cols_positivas:
    negativos = (df_raw[col] < 0).sum()
    print(f"  {col}: {negativos} valores negativos")

# Verificar que elapsed sea razonable (no ceros que distorsionen)
ceros_elapsed = (df_raw['elapsed'] == 0).sum()
print(f"\nRegistros con elapsed = 0: {ceros_elapsed}")

  elapsed: 0 valores negativos
  pi: 0 valores negativos
  po: 0 valores negativos
  speed: 0 valores negativos

Registros con elapsed = 0: 0


### 3.5 Crear columna de fecha para análisis temporal

In [14]:
# Extraer la fecha (sin hora) para agrupaciones diarias
df_raw['fecha'] = df_raw['interval_start'].dt.date

print(f"Días cubiertos en el dataset: {df_raw['fecha'].nunique()}")
print(f"Desde {df_raw['fecha'].min()} hasta {df_raw['fecha'].max()}")

Días cubiertos en el dataset: 729
Desde 2020-01-01 hasta 2022-01-01


### 3.6 Resumen de datos limpios

A continuación se presenta un resumen del dataset después de todas las transformaciones.

In [15]:
print("=" * 50)
print("RESUMEN DEL DATASET LIMPIO")
print("=" * 50)
print(f"Filas totales: {df_raw.shape[0]:,}")
print(f"Columnas: {df_raw.shape[1]}")
print(f"Rango temporal: {df_raw['interval_start'].min()} → {df_raw['interval_start'].max()}")
print(f"Equipos: {sorted(df_raw['equipo_nombre'].unique())}")
print(f"Estados: {sorted(df_raw['tipo_estado'].unique())}")
print(f"Valores nulos restantes: {df_raw.isnull().sum().sum()}")
print(f"\nEl dataset está limpio y listo para el análisis en el Notebook 02.")

RESUMEN DEL DATASET LIMPIO
Filas totales: 429,394
Columnas: 13
Rango temporal: 2020-01-01 00:05:02.863000+00:00 → 2022-01-01 23:41:35.677000+00:00
Equipos: ['Máquina 1', 'Máquina 2', 'Máquina 3', 'Máquina 4', 'Máquina 5']
Estados: ['Falla de Equipo (Downtime)', 'Inactividad (Idle)', 'Mantenimiento Planeado', 'Producción', 'Pérdida de Rendimiento']
Valores nulos restantes: 0

El dataset está limpio y listo para el análisis en el Notebook 02.
